In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.4 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2024-10-15 08:45:49,062] A new study created in memory with name: no-name-4ea45c69-006d-404c-ad54-43e7212926a3
[I 2024-10-15 08:45:51,134] Trial 0 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 153, 'max_depth': 14}. Best is trial 0 with value: 0.7653631284916201.
[I 2024-10-15 08:45:52,755] Trial 1 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 159, 'max_depth': 4}. Best is trial 0 with value: 0.7653631284916201.
[I 2024-10-15 08:45:54,814] Trial 2 finished with value: 0.7746741154562384 and parameters: {'n_estimators': 174, 'max_depth': 12}. Best is trial 2 with value: 0.7746741154562384.
[I 2024-10-15 08:45:57,031] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 181, 'max_depth': 20}. Best is trial 2 with value: 0.7746741154562384.
[I 2024-10-15 08:45:58,632] Trial 4 finished with value: 0.7746741154562383 and parameters: {'n_estimators': 137, 'max_depth': 20}. Best is trial 2 with value: 0.77467

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 198, 'max_depth': 16}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-10-15 08:46:54,026] A new study created in memory with name: no-name-fe8f0ac6-8306-47a4-8f79-5e18663cabf8
[I 2024-10-15 08:46:54,432] Trial 0 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 64, 'max_depth': 16}. Best is trial 0 with value: 0.7783985102420857.
[I 2024-10-15 08:46:54,901] Trial 1 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 81, 'max_depth': 13}. Best is trial 0 with value: 0.7783985102420857.
[I 2024-10-15 08:46:55,930] Trial 2 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 185, 'max_depth': 14}. Best is trial 0 with value: 0.7783985102420857.
[I 2024-10-15 08:46:56,893] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 172, 'max_depth': 14}. Best is trial 0 with value: 0.7783985102420857.
[I 2024-10-15 08:46:58,373] Trial 4 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 195, 'max_depth': 8}. Best is trial 0 with value: 0.7783985

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 20}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-10-15 08:47:44,761] A new study created in memory with name: no-name-9c83ba2c-17fc-48c1-8f08-3e93335ade22
[I 2024-10-15 08:47:45,328] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-10-15 08:47:46,171] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-10-15 08:47:46,464] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-10-15 08:47:47,023] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-10-15 08:47:47,588] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-10-15 08:47:59,162] A new study created in memory with name: no-name-6b996491-d7f5-4e98-9e0c-6d2d10165bc5
[I 2024-10-15 08:48:02,098] Trial 0 finished with value: 0.7318435754189944 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 149, 'learning_rate': 0.09457139050762262, 'max_depth': 11, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.7318435754189944.
[I 2024-10-15 08:48:05,634] Trial 1 finished with value: 0.756052141527002 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 181, 'learning_rate': 0.0420245458978772, 'max_depth': 9, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 1 with value: 0.756052141527002.
[I 2024-10-15 08:48:07,258] Trial 2 finished with value: 0.7765363128491621 and parameters: {'classifier': 'RandomForest', 'n_estimators': 246, 'max_depth': 8, 'min_samples_split': 6, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 2 with value: 0.7765363128491621.
[I 2024-10-15 08

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12071163672771132, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.731844,2024-10-15 08:47:59.166702,2024-10-15 08:48:02.097857,0 days 00:00:02.931155,NaN,NaN,GradientBoosting,NaN,NaN,0.094571,11.0,3.0,6.0,149.0,COMPLETE
1,1,0.756052,2024-10-15 08:48:02.100017,2024-10-15 08:48:05.633459,0 days 00:00:03.533442,NaN,NaN,GradientBoosting,NaN,NaN,0.042025,9.0,2.0,3.0,181.0,COMPLETE
2,2,0.776536,2024-10-15 08:48:05.636681,2024-10-15 08:48:07.257404,0 days 00:00:01.620723,NaN,False,RandomForest,NaN,NaN,NaN,8.0,5.0,6.0,246.0,COMPLETE
3,3,0.722533,2024-10-15 08:48:07.263181,2024-10-15 08:48:09.220586,0 days 00:00:01.957405,NaN,NaN,GradientBoosting,NaN,NaN,0.093047,16.0,3.0,8.0,66.0,COMPLETE
4,4,0.752328,2024-10-15 08:48:09.222427,2024-10-15 08:48:10.345862,0 days 00:00:01.123435,NaN,True,RandomForest,NaN,NaN,NaN,10.0,7.0,10.0,216.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.744879,2024-10-15 08:48:37.585157,2024-10-15 08:48:39.662601,0 days 00:00:02.077444,NaN,NaN,GradientBoosting,NaN,NaN,0.145426,10.0,7.0,3.0,170.0,COMPLETE
96,96,0.789572,2024-10-15 08:48:39.664552,2024-10-15 08:48:39.704604,0 days 00:00:00.040052,0.152683,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2024-10-15 08:48:39.707909,2024-10-15 08:48:39.757650,0 days 00:00:00.049741,3.410783,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.757914,2024-10-15 08:48:39.759570,2024-10-15 08:48:39.805410,0 days 00:00:00.045840,0.251919,NaN,SVM,scale,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
GradientBoosting,11
RandomForest,10


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.748603
RandomForest,0.768901
SVM,0.776607


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
!pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.3 MB/s eta 0:00:00


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2024-10-15 08:49:46,937] A new study created in memory with name: no-name-be087028-03de-44a1-b721-9a519e034bf9


[0]	train-mlogloss:0.81858	eval-mlogloss:0.83443
[1]	train-mlogloss:0.62886	eval-mlogloss:0.63561
[2]	train-mlogloss:0.48842	eval-mlogloss:0.48014
[3]	train-mlogloss:0.39164	eval-mlogloss:0.37562
[4]	train-mlogloss:0.31776	eval-mlogloss:0.29540
[5]	train-mlogloss:0.26012	eval-mlogloss:0.22454
[6]	train-mlogloss:0.22045	eval-mlogloss:0.18276
[7]	train-mlogloss:0.18920	eval-mlogloss:0.15605
[8]	train-mlogloss:0.16757	eval-mlogloss:0.13357
[9]	train-mlogloss:0.15208	eval-mlogloss:0.11558
[10]	train-mlogloss:0.14117	eval-mlogloss:0.10528
[11]	train-mlogloss:0.12693	eval-mlogloss:0.08179
[12]	train-mlogloss:0.12298	eval-mlogloss:0.07538
[13]	train-mlogloss:0.11699	eval-mlogloss:0.06599
[14]	train-mlogloss:0.11525	eval-mlogloss:0.06403
[15]	train-mlogloss:0.11194	eval-mlogloss:0.06047
[16]	train-mlogloss:0.10963	eval-mlogloss:0.05760
[17]	train-mlogloss:0.10813	eval-mlogloss:0.05516
[18]	train-mlogloss:0.10716	eval-mlogloss:0.05240
[19]	train-mlogloss:0.10281	eval-mlogloss:0.04794
[20]	train

[I 2024-10-15 08:49:48,882] Trial 0 finished with value: 1.0 and parameters: {'lambda': 1.0111482773090117e-07, 'alpha': 0.04706373391916194, 'eta': 0.24792673541725777, 'gamma': 3.112866142089435e-07, 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.4902213264354834, 'colsample_bytree': 0.8967390740854699}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:1.08430	eval-mlogloss:1.08853
[1]	train-mlogloss:1.07234	eval-mlogloss:1.07755
[2]	train-mlogloss:1.04607	eval-mlogloss:1.04909
[3]	train-mlogloss:1.03144	eval-mlogloss:1.03629
[4]	train-mlogloss:1.01090	eval-mlogloss:1.01439
[5]	train-mlogloss:0.98749	eval-mlogloss:0.98956
[6]	train-mlogloss:0.97169	eval-mlogloss:0.97271
[7]	train-mlogloss:0.94634	eval-mlogloss:0.94546
[8]	train-mlogloss:0.92951	eval-mlogloss:0.93102
[9]	train-mlogloss:0.91122	eval-mlogloss:0.91088
[10]	train-mlogloss:0.89318	eval-mlogloss:0.89279
[11]	train-mlogloss:0.87810	eval-mlogloss:0.87728
[12]	train-mlogloss:0.86093	eval-mlogloss:0.85937
[13]	train-mlogloss:0.84877	eval-mlogloss:0.84760
[14]	train-mlogloss:0.83843	eval-mlogloss:0.83856
[15]	train-mlogloss:0.82558	eval-mlogloss:0.82577
[16]	train-mlogloss:0.81416	eval-mlogloss:0.81345
[17]	train-mlogloss:0.80798	eval-mlogloss:0.81007
[18]	train-mlogloss:0.78820	eval-mlogloss:0.78957
[19]	train-mlogloss:0.77801	eval-mlogloss:0.77934
[20]	train

[I 2024-10-15 08:49:56,061] Trial 1 finished with value: 1.0 and parameters: {'lambda': 1.0660504863082756e-05, 'alpha': 2.2407555438140258e-07, 'eta': 0.02335916529785323, 'gamma': 0.059161811752074744, 'max_depth': 3, 'min_child_weight': 1, 'subsample': 0.4725625385689246, 'colsample_bytree': 0.47107449406497526}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:0.96998	eval-mlogloss:0.99162
[1]	train-mlogloss:0.88080	eval-mlogloss:0.90959


[I 2024-10-15 08:50:15,186] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:1.07495	eval-mlogloss:1.07798
[1]	train-mlogloss:1.05428	eval-mlogloss:1.05988
[2]	train-mlogloss:1.00557	eval-mlogloss:1.00729
[3]	train-mlogloss:0.98189	eval-mlogloss:0.98499


[I 2024-10-15 08:50:15,659] Trial 49 pruned. Trial was pruned at iteration 4.


Best trial: {'lambda': 1.0111482773090117e-07, 'alpha': 0.04706373391916194, 'eta': 0.24792673541725777, 'gamma': 3.112866142089435e-07, 'max_depth': 8, 'min_child_weight': 2, 'subsample': 0.4902213264354834, 'colsample_bytree': 0.8967390740854699}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()